In [1]:
%run -m ipy_startup
%run -m ipy_logging
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import data_type as dtyp
from mgds.data_aggregation import io_utils as io
from mgds.data_aggregation import entity
from py_utils import set_utils, assertion_utils
pd.set_option('display.max_info_rows', 50000000)

In [34]:
sources = [src.GDSC_v2, src.CCLE_v1, src.NCI60_v2, src.NCIDREAM_v1, src.CTD_v2]
data_types = [
    dtyp.GENE_EXOME_SEQ, dtyp.GENE_EXPRESSION, dtyp.GENE_COPY_NUMBER, 
    dtyp.GENE_RNA_SEQ, dtyp.GENE_METHYLATION, 
    dtyp.CELLLINE_META, dtyp.DRUG_SENSITIVITY
]
m_id = entity.get_raw_entities(sources, data_types, 'CELL_LINE_ID')

2016-12-01 12:53:39,528:DEBUG:mgds.data_aggregation.entity: Processing source "gdsc_v2", data type "gene-exome-seq"
2016-12-01 12:53:39,842:DEBUG:mgds.data_aggregation.entity: Processing source "gdsc_v2", data type "gene-expression"
2016-12-01 12:53:46,896:DEBUG:mgds.data_aggregation.entity: Processing source "gdsc_v2", data type "gene-exome-seq"
2016-12-01 12:53:47,213:DEBUG:mgds.data_aggregation.entity: Processing source "gdsc_v2", data type "gene-rna-seq"
2016-12-01 12:53:47,214:INFO:mgds.data_aggregation.entity: Data for source "gdsc_v2" and data type "gene-rna-seq" does not exist
2016-12-01 12:53:47,214:DEBUG:mgds.data_aggregation.entity: Processing source "gdsc_v2", data type "gene-methylation"
2016-12-01 12:53:47,215:INFO:mgds.data_aggregation.entity: Data for source "gdsc_v2" and data type "gene-methylation" does not exist
2016-12-01 12:53:47,215:DEBUG:mgds.data_aggregation.entity: Processing source "gdsc_v2", data type "cellline-meta"
2016-12-01 12:53:47,222:DEBUG:mgds.data_ag

In [35]:
# dt = m_id[src.CCLE_v1]['gene-expression']
# dt[dt['CELL_LINE_ID'].str.contains('TT')]

In [36]:
import re
regex = re.compile('[\W_]+')
def clean(x):
    return regex.sub('', x)

def to_mgds_id(source, v):
    fn = None
    if pd.isnull(v):
        return None
    
    # NCI-60 Special Cases
    if source == src.NCI60_v2:
        return clean(v).upper().strip()
    
    # CTD Special Cases
    if source == src.CTD_v2:
        return clean(v).upper().strip()
    
    # GDSC/COSMIC Special Cases
    if source == src.GDSC_v2:
        v = v.upper().strip()
        if v == 'T-T':
            return 'TTOESOPHAGUS'
        if v == 'TT':
            return 'TTTHYROID'
        if v == 'KM-H2':
            return 'KMH2BLOOD'
        if v == 'KMH-2':
            return 'KMH2THYR'
        if v == '786-0':
            return '786O'
        if v == 'EOL-1-CELL':
            return 'EOL1'
        return clean(v).strip()
    
    # CCLE Special Cases
    if source == src.CCLE_v1:
        v = v.upper().strip()
        if v == 'TT_OESOPHAGUS':
            return 'TTOESOPHAGUS'
        if v == 'TT_THYROID':
            return 'TTTHYROID'
        return v.split('_')[0].strip()
    
    if source == src.NCIDREAM_v1:
        return clean(v).strip()

    raise ValueError('Cell line normalization for source "{}" has not been implemented yet'.format(source))

def to_mgds_ids(source, ids):
    return [to_mgds_id(source, v) for v in ids]


def aggregate(m_id):
    r = []
    for source in m_id.keys():
        for data_type, d in m_id[source].items():
            c_cl = d.filter(regex='CELL_LINE_ID').columns.tolist()
            d_id = d.copy().assign(MGDS_ID=to_mgds_ids(source, d['CELL_LINE_ID']))
            for c in c_cl:
                taxonomy = c.split(':')[1] if ':' in c else 'COMMON'
                d_pt = d_id.assign(TAXONOMY=taxonomy)[['MGDS_ID', 'DATA_TYPE', 'SOURCE', 'TAXONOMY', c]]
                d_pt = d_pt.rename(columns={c: 'CELL_LINE_ID'})
                r.append(d_pt)
    return pd.concat(r).reset_index(drop=True)

In [37]:
d_id = aggregate(m_id)

In [38]:
d_id.head()

,MGDS_ID,DATA_TYPE,SOURCE,TAXONOMY,CELL_LINE_ID
0,600MPE,gene-methylation,ncidream_v1,COMMON,600MPE
1,AU565,gene-methylation,ncidream_v1,COMMON,AU565
2,BT20,gene-methylation,ncidream_v1,COMMON,BT20
3,BT474,gene-methylation,ncidream_v1,COMMON,BT474
4,BT483,gene-methylation,ncidream_v1,COMMON,BT483


In [41]:
# d_id[d_id['SOURCE'] == src.NCI60_v2]['CELL_LINE_ID'].str.contains(':').value_counts()

In [42]:
# This should return no results when there are no duplicates
cts = d_id.groupby(['SOURCE', 'DATA_TYPE', 'TAXONOMY', 'MGDS_ID']).size()
print(cts.value_counts())
cts[cts > 1]

1    13115
dtype: int64


Series([], dtype: int64)

In [7]:
# d_id[(d_id['SOURCE'] == src.GDSC_v2) & (d_id['MGDS_ID'] == 'KMH2')]

In [43]:
def singlestr(x):
    assert len(x) == 1
    return x.iloc[0]
d_id_m = d_id.pivot_table(index='MGDS_ID', columns=['TAXONOMY', 'SOURCE', 'DATA_TYPE'], values='CELL_LINE_ID', aggfunc=singlestr)
d_id_m.head()

TAXONOMY                          COMMON                 \
SOURCE                           ccle_v1                  
DATA_TYPE                  cellline-meta gene-exome-seq   
MGDS_ID                                                   
1321N1     1321N1_CENTRAL_NERVOUS_SYSTEM           None   
143B                           143B_BONE           None   
184A1                               None           None   
184B5                               None           None   
2004                                None           None   

TAXONOMY                                                                 \
SOURCE                                             ctd_v2       gdsc_v2   
DATA_TYPE                gene-expression drug-sensitivity cellline-meta   
MGDS_ID                                                                   
1321N1     1321N1_CENTRAL_NERVOUS_SYSTEM             None          None   
143B                           143B_BONE             None          None   
184A1                               None             None          None   
184B5                               None             None          None   
2004                                None             2004          None   

TAXONOMY                                                                 \
SOURCE                                                         nci60_v2   
DATA_TYPE drug-sensitivity gene-exome-seq gene-expression cellline-meta   
MGDS_ID                                                                   
1321N1                None           None            None          None   
143B                  None           None            None          None   
184A1                 None           None            None          None   
184B5                 None           None            None          None   
2004                  None           None            None          None   

TAXONOMY                         ...                                        \
SOURCE                           ...                           ncidream_v1   
DATA_TYPE drug-sensitivity       ...        gene-expression gene-exome-seq   
MGDS_ID                          ...                                         
1321N1                None       ...                   None           None   
143B                  None       ...                   None           None   
184A1                 None       ...                   None          184A1   
184B5                 None       ...                   None          184B5   
2004                  None       ...                   None           None   

TAXONOMY                                                       COSMIC  \
SOURCE                                                        gdsc_v2   
DATA_TYPE gene-expression gene-methylation gene-rna-seq cellline-meta   
MGDS_ID                                                                 
1321N1               None             None         None          None   
143B                 None             None         None          None   
184A1                None             None        184A1          None   
184B5               184B5             None        184B5          None   
2004                 None             None         None          None   

TAXONOMY                                                               CTD  
SOURCE                                                              ctd_v2  
DATA_TYPE drug-sensitivity gene-exome-seq gene-expression drug-sensitivity  
MGDS_ID                                                                     
1321N1                None           None            None             None  
143B                  None           None            None             None  
184A1                 None           None            None             None  
184B5                 None           None            None             None  
2004                  None           None            None           155823  

[5 rows x 21 columns]

In [44]:
d_id_m.sort_index().head(50).tail(51)

TAXONOMY                                          COMMON  \
SOURCE                                           ccle_v1   
DATA_TYPE                                  cellline-meta   
MGDS_ID                                                    
1321N1                     1321N1_CENTRAL_NERVOUS_SYSTEM   
143B                                           143B_BONE   
184A1                                               None   
184B5                                               None   
2004                                                None   
201T                                                None   
21MT1                                               None   
21NT                                                None   
22RV1                                     22RV1_PROSTATE   
2313287                                  2313287_STOMACH   
253J                                  253J_URINARY_TRACT   
253JBV                              253JBV_URINARY_TRACT   
42MGBA                     42MGBA_CENTRAL_NERVOUS_SYSTEM   
451LU                                               None   
5637                                  5637_URINARY_TRACT   
59M                                            59M_OVARY   
600MPE                                              None   
639V                                  639V_URINARY_TRACT   
647V                                  647V_URINARY_TRACT   
697               697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
769P                                         769P_KIDNEY   
7860                                                None   
786O                                         786O_KIDNEY   
8305C                                      8305C_THYROID   
8505C                                      8505C_THYROID   
8MGBA                       8MGBA_CENTRAL_NERVOUS_SYSTEM   
A101D                                         A101D_SKIN   
A172                         A172_CENTRAL_NERVOUS_SYSTEM   
A204                                    A204_SOFT_TISSUE   
A2058                                         A2058_SKIN   
A253                                 A253_SALIVARY_GLAND   
A2780                                        A2780_OVARY   
A375                                           A375_SKIN   
A388                                                None   
A3KAW           A3KAW_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
A427                                                None   
A431                                                None   
A498                                         A498_KIDNEY   
A4FUK           A4FUK_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
A549                                           A549_LUNG   
A549ATCC                                            None   
A673                                           A673_BONE   
A704                                         A704_KIDNEY   
ABC1                                           ABC1_LUNG   
ACCMESO1                                 ACCMESO1_PLEURA   
ACCS                                  ACCS_URINARY_TRACT   
ACHN                                         ACHN_KIDNEY   
ACN                                                 None   
AGS                                          AGS_STOMACH   
ALEXANDERCELLS                      ALEXANDERCELLS_LIVER   

TAXONOMY                                                  \
SOURCE                                                     
DATA_TYPE                                 gene-exome-seq   
MGDS_ID                                                    
1321N1                                              None   
143B                                                None   
184A1                                               None   
184B5                                               None   
2004                                                None   
201T                                                None   
21MT1                                               None   
21NT                                                None   
22RV1                                  

In [45]:
d_id_m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1479 entries, 1321N1 to ZR75B
Data columns (total 21 columns):
(COMMON, ccle_v1, cellline-meta)           1019 non-null object
(COMMON, ccle_v1, gene-exome-seq)          904 non-null object
(COMMON, ccle_v1, gene-expression)         967 non-null object
(COMMON, ctd_v2, drug-sensitivity)         887 non-null object
(COMMON, gdsc_v2, cellline-meta)           1029 non-null object
(COMMON, gdsc_v2, drug-sensitivity)        1012 non-null object
(COMMON, gdsc_v2, gene-exome-seq)          1001 non-null object
(COMMON, gdsc_v2, gene-expression)         976 non-null object
(COMMON, nci60_v2, cellline-meta)          60 non-null object
(COMMON, nci60_v2, drug-sensitivity)       60 non-null object
(COMMON, nci60_v2, gene-exome-seq)         60 non-null object
(COMMON, nci60_v2, gene-expression)        54 non-null object
(COMMON, ncidream_v1, gene-exome-seq)      50 non-null object
(COMMON, ncidream_v1, gene-expression)     46 non-null object
(COMMON, nci

In [46]:
from mgds.data_aggregation import excel_utils
excel_utils.is_excel_date(d_id['CELL_LINE_ID']).value_counts()

False    11327
Name: CELL_LINE_ID, dtype: int64

## Questionable Matches

- **GP2D** <-> **GP5D**: ccle_v1/GP2D_LARGE_INTESTINE <-> gdsc_v2/GP5d (large intenstine site)
- **HEY** <-> **HEYA8**: ccle_v1/HEYA8_OVARY <-> gdsc_v2/Hey (ovary site)
- **U251** <-> **U251MG**: ccle_v1/U251MG_CENTRAL_NERVOUS_SYSTEM <-> gdsc_v2/U251 (central_nervous_system)
- **U266** <-> **U266B1**: ccle_v1/U266B1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE <-> gdsc_v2/U-266 (haematopoietic_and_lymphoid_tissue)
- **LU99** <-> **LU99A**: ccle_v1/LU99_LUNG <-> gdsc_v2/LU-99A



In [23]:
check_idx = []
for i in range(len(d_id_m)-1):
    v1 = d_id_m.index.values[i]
    v2 = d_id_m.index.values[i+1]
    if v1 in v2:
        check_idx.append(i)
        check_idx.append(i+1)
d_id_m.iloc[check_idx, :].head(500).tail(51)

TAXONOMY                                       COMMON  \
SOURCE                                        ccle_v1   
DATA_TYPE                               cellline-meta   
MGDS_ID                                                 
RT11284                         RT11284_URINARY_TRACT   
SCC9                   SCC9_UPPER_AERODIGESTIVE_TRACT   
SCC90                                            None   
SKMEL2                                    SKMEL2_SKIN   
SKMEL24                                  SKMEL24_SKIN   
SKMEL3                                    SKMEL3_SKIN   
SKMEL30                                  SKMEL30_SKIN   
SKN                                              None   
SKN3                                             None   
SNU1                                     SNU1_STOMACH   
SNU1033                       SNU1033_LARGE_INTESTINE   
SNU119                                   SNU119_OVARY   
SNU1196                         SNU1196_BILIARY_TRACT   
SNU46                 SNU46_UPPER_AERODIGESTIVE_TRACT   
SNU466                  SNU466_CENTRAL_NERVOUS_SYSTEM   
SNU5                                     SNU5_STOMACH   
SNU503                         SNU503_LARGE_INTESTINE   
SNU8                                       SNU8_OVARY   
SNU81                           SNU81_LARGE_INTESTINE   
SR                                               None   
SR786        SR786_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
SUDHL1      SUDHL1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
SUDHL10    SUDHL10_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
SUPT1        SUPT1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
SUPT11      SUPT11_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
SW13                                             None   
SW1353                                    SW1353_BONE   
SW48                             SW48_LARGE_INTESTINE   
SW480                           SW480_LARGE_INTESTINE   
T47D                                      T47D_BREAST   
T47DKBluc                                        None   
TE1                                    TE1_OESOPHAGUS   
TE10                                  TE10_OESOPHAGUS   
TE12                                             None   
TE125T                             TE125T_SOFT_TISSUE   
TE15                                  TE15_OESOPHAGUS   
TE159T                             TE159T_SOFT_TISSUE   
TE4                                    TE4_OESOPHAGUS   
TE441T                             TE441T_SOFT_TISSUE   
TE6                                    TE6_OESOPHAGUS   
TE617T                             TE617T_SOFT_TISSUE   
TK                                               None   
TK10                                             None   
U251                                             None   
U251MG                  U251MG_CENTRAL_NERVOUS_SYSTEM   
U266                                             None   
U266B1      U266B1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
WM793                                      WM793_SKIN   
WM793B                                           None   
YMB1                                      YMB1_BREAST   
YMB1E                                            None   

TAXONOMY                                               \
SOURCE                                                  
DATA_TYPE                              gene-exome-seq   
MGDS_ID                                                 
RT11284                         RT11284_URINARY_TRACT   
SCC9                   SCC9_UPPER_AERODIGESTIVE_TRACT   
SCC90                                            None   
SKMEL2                                    SKMEL2_SKIN   
SKMEL24                                  SKMEL24_SKIN   
SKMEL3                                    SKMEL3_SKIN   
SKMEL30                                  SKMEL30_SKIN   
SKN                                              None   
SKN3                                             None   
SNU1                                     SNU1_STOMACH   
SNU1033                       SNU1033_LARGE_INTESTINE   
SNU119   

## Match Analysis

In [24]:
d_id_m.loc['U266':].head(3)

TAXONOMY                                      COMMON  \
SOURCE                                       ccle_v1   
DATA_TYPE                           gene-copy-number   
MGDS_ID                                                
U266                                            None   
U266B1     U266B1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
U2OS                                       U2OS_BONE   

TAXONOMY                                              \
SOURCE                                                 
DATA_TYPE                             gene-exome-seq   
MGDS_ID                                                
U266                                            None   
U266B1     U266B1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE   
U2OS                                       U2OS_BONE   

TAXONOMY                                                               \
SOURCE                                                        gdsc_v2   
DATA_TYPE                            gene-expression gene-copy-number   
MGDS_ID                                                                 
U266                                            None            U-266   
U266B1     U266B1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE             None   
U2OS                                       U2OS_BONE           U-2-OS   

TAXONOMY                                                                  \
SOURCE                                           nci60_v2                  
DATA_TYPE gene-exome-seq gene-expression gene-copy-number gene-exome-seq   
MGDS_ID                                                                    
U266               U-266           U-266             None           None   
U266B1              None            None             None           None   
U2OS              U-2-OS          U-2-OS             None           None   

TAXONOMY                                                                   \
SOURCE                         ncidream_v1                                  
DATA_TYPE gene-expression gene-copy-number gene-exome-seq gene-expression   
MGDS_ID                                                                     
U266                 None             None           None            None   
U266B1               None             None           None            None   
U2OS                 None             None           None            None   

TAXONOMY                                          COSMIC                 \
SOURCE                                           gdsc_v2                  
DATA_TYPE gene-methylation gene-rna-seq gene-copy-number gene-exome-seq   
MGDS_ID                                                                   
U266                  None         None           753615         753615   
U266B1                None         None             None           None   
U2OS                  None         None           909776         909776   

TAXONOMY                   
SOURCE                     
DATA_TYPE gene-expression  
MGDS_ID                    
U266               753615  
U266B1               None  
U2OS               909776

In [19]:
d_gdsc = db.load(src.GDSC_v2, db.IMPORT, 'cellline-meta')
d_gdsc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 0 to 1033
Data columns (total 15 columns):
CELL_LINE_ID                         1029 non-null object
CELL_LINE_ID:COSMIC                  1029 non-null object
WHOLE_EXOME_SEQUENCING               1001 non-null object
COPY_NUMBER_ALTERATIONS              1001 non-null object
GENE_EXPRESSION                      1001 non-null object
METHYLATION                          1001 non-null object
DRUGRESPONSE                         1001 non-null object
GDSC_TISSUE_DESCRIPTOR_1             1001 non-null object
GDSC_TISSUE_DESCRIPTOR_2             1001 non-null object
CANCER_TYPE                          826 non-null object
MICROSATELLITE_INSTABILITY_STATUS    986 non-null object
SCREEN_MEDIUM                        1001 non-null object
GROWTH_PROPERTIES                    999 non-null object
SITE                                 1024 non-null object
HISTOLOGY                            1024 non-null object
dtypes: object(15)
memory 

In [30]:
d_gdsc[d_gdsc['CELL_LINE_ID:COSMIC'] == '907796']

,CELL_LINE_ID,CELL_LINE_ID:COSMIC,WHOLE_EXOME_SEQUENCING,COPY_NUMBER_ALTERATIONS,GENE_EXPRESSION,METHYLATION,DRUGRESPONSE,GDSC_TISSUE_DESCRIPTOR_1,GDSC_TISSUE_DESCRIPTOR_2,CANCER_TYPE,MICROSATELLITE_INSTABILITY_STATUS,SCREEN_MEDIUM,GROWTH_PROPERTIES,SITE,HISTOLOGY
518,LU-99A,907796,Y,Y,Y,Y,Y,lung_NSCLC,lung_NSCLC_large cell,NaN,MSS/MSI-L,R,Adherent,lung,carcinoma


# Compress to Source

In [47]:
d_id.head()

,MGDS_ID,DATA_TYPE,SOURCE,TAXONOMY,CELL_LINE_ID
0,600MPE,gene-methylation,ncidream_v1,COMMON,600MPE
1,AU565,gene-methylation,ncidream_v1,COMMON,AU565
2,BT20,gene-methylation,ncidream_v1,COMMON,BT20
3,BT474,gene-methylation,ncidream_v1,COMMON,BT474
4,BT483,gene-methylation,ncidream_v1,COMMON,BT483


In [48]:
def get_unique_id(g):
    ids = g['CELL_LINE_ID'].astype(str)
    assert len(np.unique(ids)), 'Found group with conflicting ids across data types:\n{}'.format(g)
    return ids.iloc[0]

# Get identifier for each source + taxonomy by first making sure there are no conflicts across data types
d_id_src = d_id.groupby(['MGDS_ID', 'SOURCE', 'TAXONOMY']).apply(get_unique_id)
d_id_src = d_id_src.unstack().unstack()

# Make sure all IDs are represented as strings
assertion_utils.assert_object_types(d_id_src)

d_id_src.head()

TAXONOMY                         COMMON                                      \
SOURCE                          ccle_v1 ctd_v2 gdsc_v2 nci60_v2 ncidream_v1   
MGDS_ID                                                                       
1321N1    1321N1_CENTRAL_NERVOUS_SYSTEM   None    None     None        None   
143B                          143B_BONE   None    None     None        None   
184A1                              None   None    None     None       184A1   
184B5                              None   None    None     None       184B5   
2004                               None   2004    None     None        None   

TAXONOMY  COSMIC                                         CTD                  \
SOURCE   ccle_v1 ctd_v2 gdsc_v2 nci60_v2 ncidream_v1 ccle_v1  ctd_v2 gdsc_v2   
MGDS_ID                                                                        
1321N1      None   None    None     None        None    None    None    None   
143B        None   None    None     None        None    None    None    None   
184A1       None   None    None     None        None    None    None    None   
184B5       None   None    None     None        None    None    None    None   
2004        None   None    None     None        None    None  155823    None   

TAXONOMY                       
SOURCE   nci60_v2 ncidream_v1  
MGDS_ID                        
1321N1       None        None  
143B         None        None  
184A1        None        None  
184B5        None        None  
2004         None        None

# Export

In [49]:
db.save(d_id_src, src.MGDS_v1, db.ENTITY, 'cellline-ids-by-src')

'/Users/eczech/data/research/mgds/entity/mgds_v1_cellline-ids-by-src.pkl'

In [50]:
db.save(d_id_m, src.MGDS_v1, db.ENTITY, 'cellline-ids-by-typ')

'/Users/eczech/data/research/mgds/entity/mgds_v1_cellline-ids-by-typ.pkl'